# Script execution actions

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
  * Action `script` is a general script execution action
  * Actions such as `Python2`, `Python3`, `Perl` executes scripts with specified interpreters
  

## Action `script`

Action `script` is the general form of all script-executing actions in SoS. It accepts a script, and parameters `interpreter` (required), `suffix` (if required by the interpreter) and optional `args` (command line arguments). It can be used to execute any script for which its interpreter is not currently supported by SoS. For example, the action

```
python:
    print('HERE')
```

can be executed as

In [1]:
script: interpreter='python'
    print('HERE')

HERE


## Action  `perl`

Action `perl` executes the passed script using `perl` interpreter.

In [2]:
perl:
    my $name = "Brian";
    print "Hello, $name!\n";

Hello, Brian!


## Action  `ruby`

Action `ruby` execute the passed script using `ruby` interpreter.

In [3]:
ruby:
    a = [ 45, 3, 19, 8 ]
    b = [ 'sam', 'max', 56, 98.9, 3, 10, 'jill' ]
    print (a + b).join(' '), "\n"
    print a[2], " ", b[4], " ", b[-2], "\n"
    print a.sort.join(' '), "\n"
    a << 57 << 9 << 'phil'
    print "A: ", a.join(' '), "\n"

/System/Library/Frameworks/Ruby.framework/Versions/2.3/usr/lib/ruby/2.3.0/universal-darwin18/rbconfig.rb:215: warning: Insecure world writable dir /Applications/FastQC.app/Contents/MacOS in PATH, mode 040777
45 3 19 8 sam max 56 98.9 3 10 jill
19 3 10
3 8 19 45
A: 45 3 19 8 57 9 phil


##  Action  `node`

Action `node` executes the passed script using `node` (JavaScript) interpreter.

In [4]:
node:
    var i, a, b, c, max;

    max = 1000000000;

    var d = Date.now();

    for (i = 0; i < max; i++) {
        a = 1234 + 5678 + i;
        b = 1234 * 5678 + i;
        c = 1234 / 2 + i;
    }

    console.log(Date.now() - d);

967


## Action `julia`

Action `julia` execute specified Julia script with command `julia`. Because my system does not have julia installed, the following example executes a Julia script in a docker container.

In [5]:
julia: container='julia'
    s1 = "The quick brown fox jumps over the lazy dog α,β,γ"
    println(s1)

Pulling docker image julia


The quick brown fox jumps over the lazy dog α,β,γ


### Action `matlab`

Action `matlab` execute specified matlab script with default options `-nojvm -nodisplay -nosplash -r {filename:n};quit`. The options start matlab without desktop, execute the strip (with parameter without `.m` extension), and quit matlab when the script is completed.

### Action `python`

Action `python` and `python3` accepts a Python script and execute it with python or python3, respectively.

Because SoS can include Python statements directly in a SoS script, it is important to note that embedded Python
statements are interpreted by SoS and the `python` and `python3` actions are execute in separate processes without access to the SoS environment.

For example, the following SoS step execute some python statements **within** SoS with direct access to SoS variables such as `input`, and with `result` writing directly to the SoS environment,

```python
[10]
for filename in input:
    with open(filename) as data:
        result = filename + '.res'
        ....
```

while

```python
[10]
input: group_by='single'

python:

with open(${input!r}) as data:
   result = ${input!r} + '.res'
   ...


```

composes a Python script for each input file and calls separate Python interpreters to execute them. Whereas
the Python statement in the first example will always be executed, the statements in `python` will not be executed in `inspect` mode.

In the following example, `python` actually calls Python 3 because version 3 is the default python interpreter on my system.

In [6]:
python:
  import sys
  print(sys.version)

3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 14:38:56) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Action `python2`

Action `python2` is similar to `python` but it tries to use interpreter `python2` (or `python2.7` on some systems) before `python`, which could be python 3. Note that this action does not actually test the version of interpreter so it would use python 3 if this is the only available version.

In [7]:
python2:
  print "Hello world"

Hello world


### Action `python3`

Action `python3` is similar to `python` but it tries to use interpreter `python3` (version 3 of python) before `python`, which could be python 2. Note that this action does not actually test the version of interpreter so it would use python 2 if this is the only available version.

In [8]:
python3:
  print('Hello world')

Hello world


### Action `R`

Action `R(script)` execute the passed script using `Rscript` command. 

In [9]:
R:
    D <- data.frame(x=c(1,2,3,1), y=c(7,19,2,2))
    # Sort on x
    indexes <- order(D$x)
    D[indexes,]

  x  y
1 1  7
4 1  2
2 2 19
3 3  2


### Parameter `entrypoint`

You can use parameter entrypoint to change the environment in which the interpreter will be executed. An entrypoint is usually the name of a shell script that defines a number of environment variables and run the command given by the command line parameters in such a way that the current process is replaced by it,

```SoS
#!/bin/bash

set -o errexit
set -o pipefail
set -o nounset

export MY_ENV=MY_VALUE

exec "$@"
```
but it can also be other commands such as `micromamba run -n ENVIRONMENT`.

For example, the following statement will execute `interpreter` Rscript

In [ ]:
R: entrypoint = 'micromamba run -n ENVIRONMENT'

INFO:    Running default:
INFO:    default is completed.
INFO:    Workflow default is executed successfully with 1 completed step.


It's equivalent to the following statement

In [ ]:
script: interpreter = Rscript, entrypoint = 'micromamba run -n ENVIRONMENT'